In [1]:
import waterclusters as wc
import pymol as pm
import Bio.PDB
import Bio
from Bio import SeqIO
from Bio import pairwise2
import helperfunctions as _hf


from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.Polypeptide import PPBuilder
%load_ext autoreload
%autoreload 2

In [ ]:
amino_d = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

In [ ]:
amino_1l = {'C' :'CYS', 'D':'ASP', 'S':'SER', 'Q':'GLN', 'K':'LYS', 
            'I':'ILE', 'P':'PRO', 'T':'THR', 'F': 'PHE', 'N': 'ASN', 
     'G': 'GLY', 'H': 'HIS', 'L': 'LEU', 'R': 'ARG', 'W': 'TRP', 
     'A': 'ALA', 'V': 'VAL', 'E': 'GLU', 'Y': 'TYR', 'M': 'MET'}

In [2]:
pdb_root_folder = '../workfolder/test_files_GPCR/'
# pdb_root_folder = '../workfolder/test_files_GlplG/'
pdb_root_folder = '../workfolder/test_files_SARS/'
target_folder = pdb_root_folder

w = wc.WaterClusters(pdb_root_folder)
file_list = w.file_list

../workfolder/test_files_SARS/


In [3]:
target_folder

'../workfolder/test_files_SARS/'

In [4]:
w.number_of_waters_per_structure()

0
0
0
0
0


In [5]:
def get_sequence(pdb_file):
    sequence = []
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('model', pdb_file)
    ppb = PPBuilder()
    for pp in ppb.build_peptides(structure):
        sequence.append(pp.get_sequence())
    seq = ''
    for s in sequence:
        seq += s
    return seq

In [6]:
def align_sequence(pdb_ref, pdb_move, threshold=0.75):
    ref_sequence = get_sequence(pdb_ref)
    move_sequence = get_sequence(pdb_move)
    alignments = pairwise2.align.globalxx(ref_sequence, move_sequence)
    
    best_score = 0
    best_i = 0
    for i, alignment in enumerate(alignments):
        if best_score <= alignment.score:
            best_score = alignment.score
            best_i = i

    best_alginment = alignments[best_i]
    assert len(best_alginment.seqA) == len(best_alginment.seqB), 'Aligned sequences have different lenght'
    assert best_alginment.score / len(alignments[best_i].seqA) >= threshold, 'Sequences of '+pdb_move+' differ more than the threshold value ('+str(threshold*100)+'%) from the reference structure'
    assert best_alginment.score / len(alignments[best_i].seqB) >= threshold, 'Sequences of '+pdb_move+' differ more than the threshold value ('+str(threshold*100)+'%) from the reference structure'

    return best_alginment.seqA, best_alginment.seqB

In [7]:
pdb_ref = file_list[0]
for file in file_list[1:]:
    pdb_move = file
    print(pdb_move)
    ref_aligned, move_aligned = align_sequence(pdb_root_folder+pdb_ref,
                                               pdb_root_folder+pdb_move,
                                              threshold=0.75)
    _hf.superimpose_aligned_atoms(ref_aligned, pdb_root_folder+pdb_ref,
                                  move_aligned, pdb_root_folder+pdb_move,
                                  file_name=target_folder+pdb_move)

6m0jA.pdb
6m17B.pdb
6vw1A.pdb
6vw1B.pdb


* all sequence has to have a minimum threshold similarity min 75%, but can be set
* if not than meaningful error message that some of them is not compared to the reference file
* set reference file as the first one in the list but can be set manulaly